In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from keras.callbacks import LearningRateScheduler
from keras.layers import Dropout
from keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers
from kerastuner.tuners import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\Administrator\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [2]:
# 图片路径
image_folder = "E:\shujuji\metamaterialpicture900"
# Excel文件路径
excel_path = "E:\\1-1000-90.xlsx"
df = pd.read_excel(excel_path, usecols=[0, 16])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行
# 初始化图像和性能数组
X = []
y = []
for i in range(1, 651):
    img_path = os.path.join(image_folder, f"lcy{i}.jpg")
    if i in df["编号"].values:
        img = load_img(img_path, target_size=(224, 224))  # 调整为合适的图像大小
        img_array = img_to_array(img)
        img_array = cv2.cvtColor(img_array.astype('uint8'), cv2.COLOR_RGB2GRAY)
        X.append(img_array)
y=df["d33"]
# 转换为NumPy数组
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train = X_train / 255.0
X_test = X_test / 255.0

In [3]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(224, 224, 1)))
    for i in range(hp.Int('num_blocks1', 1, 3)):
        model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform', input_shape=(224, 224, 1), kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout1_'+ str(i), values=[0.0, 0.1])))
    model.add(AveragePooling2D((2, 2)))

    for i in range(hp.Int('num_blocks2', 1, 3)):
        model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout2_'+ str(i), values=[0.0, 0.1])))
    model.add(AveragePooling2D((2, 2)))

    for i in range(hp.Int('num_blocks3', 1, 3)):
        model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout3_'+ str(i), values=[0.0, 0.1])))
    model.add(AveragePooling2D((2, 2)))



    model.add(Flatten())

    for j in range(hp.Int('num_dense_layers1', 1, 3)):
        model.add(Dense(512, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout5_'+ str(j), values=[0.0, 0.1])))
    for j in range(hp.Int('num_dense_layers2', 1, 3)):
        model.add(Dense(256, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout6_'+ str(j), values=[0.0, 0.1])))
    for j in range(hp.Int('num_dense_layers3', 1, 3)):
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout7_'+ str(j), values=[0.0, 0.1])))
    for j in range(hp.Int('num_dense_layers4', 1, 3)):
        model.add(Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(hp.Choice('dropout8_'+ str(j), values=[0.0, 0.1])))


    model.add(Dense(1, activation='linear'))
    hp_learning_rate = hp.Choice('learning_rate' , values=[0.01,0.015,0.02,0.025,0.03,0.005])
    hp_optimizer=hp.Choice('Optimizer', values=['Adam', 'SGD'])
    if hp_optimizer == 'Adam':
        hp_learning_rate = hp.Choice('learning_rate' , values=[0.01,0.015,0.02,0.025,0.03,0.005])
    elif hp_optimizer == 'SGD':
        hp_learning_rate = hp.Choice('learning_rate' , values=[0.01,0.015,0.02,0.025,0.03,0.005])
        nesterov=True
        momentum=0.9

    model.compile(optimizer=hp_optimizer,loss='mean_squared_error')
    return model
tuner_cnn =BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=40,
    overwrite=True,
    directory='E:\kerast',
    project_name='tuner_cnn1')
    
tuner_cnn.search(X_train, y_train,epochs=50,batch_size=64,validation_data=(X_test, y_test))
# 打印搜索结果摘要

Trial 16 Complete [00h 38m 37s]
val_loss: nan

Best val_loss So Far: 1345.834228515625
Total elapsed time: 09h 15m 53s

Search: Running Trial #17

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_blocks1
0                 |0                 |dropout1_0
1                 |3                 |num_blocks2
0                 |0                 |dropout2_0
2                 |3                 |num_blocks3
0                 |0.1               |dropout3_0
1                 |1                 |num_dense_layers1
0                 |0.1               |dropout5_0
1                 |1                 |num_dense_layers2
0.1               |0                 |dropout6_0
3                 |2                 |num_dense_layers3
0                 |0                 |dropout7_0
3                 |1                 |num_dense_layers4
0.1               |0.1               |dropout8_0
0.01              |0.005             |learning_rate
SGD               |Adam        

RuntimeError: Number of consecutive failures exceeded the limit of 3.


In [4]:
best_cnn_hyperparameters = tuner_cnn.get_best_hyperparameters(1)[0]
print("Best Hyper-parameters")
best_cnn_hyperparameters.values
tuner_cnn.results_summary()

Best Hyper-parameters
Results summary
Results in E:\kerast\tuner_cnn1
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 08 summary
Hyperparameters:
num_blocks1: 3
dropout1_0: 0.0
num_blocks2: 3
dropout2_0: 0.0
num_blocks3: 3
dropout3_0: 0.1
num_dense_layers1: 1
dropout5_0: 0.1
num_dense_layers2: 1
dropout6_0: 0.0
num_dense_layers3: 2
dropout7_0: 0.0
num_dense_layers4: 1
dropout8_0: 0.1
learning_rate: 0.005
Optimizer: Adam
dropout1_1: 0.0
dropout1_2: 0.0
dropout5_1: 0.1
dropout5_2: 0.0
dropout6_1: 0.1
dropout6_2: 0.0
dropout7_1: 0.0
dropout7_2: 0.1
dropout8_1: 0.1
dropout8_2: 0.0
dropout2_1: 0.0
dropout2_2: 0.1
dropout3_1: 0.1
dropout3_2: 0.1
Score: 1345.834228515625

Trial 09 summary
Hyperparameters:
num_blocks1: 1
dropout1_0: 0.0
num_blocks2: 2
dropout2_0: 0.0
num_blocks3: 2
dropout3_0: 0.1
num_dense_layers1: 2
dropout5_0: 0.0
num_dense_layers2: 1
dropout6_0: 0.0
num_dense_layers3: 2
dropout7_0: 0.0
num_dense_layers4: 3
dropout8_0: 0.0
learning_rate: 0.025
Opti

In [ ]:
model_cnn = Sequential()

model_cnn.add(Input(shape=(224, 224, 1)))

for i in range(best_cnn_hyperparameters['num_blocks']):
    hp_filters=best_cnn_hyperparameters['filters_'+ str(i)]

    model_cnn.add(Conv2D(hp_filters, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(224, 224, 1)))
    model_cnn.add(AveragePooling2D((2, 2)))
    model_cnn.add(Dropout(best_cnn_hyperparameters['dropout_'+ str(i)]))

model_cnn.add(Flatten())
model_cnn.add(Dense(best_cnn_hyperparameters['units'], activation='relu', kernel_initializer='he_uniform'))
 
model.add(Dense(1, activation='linear'))

model_cnn.compile(optimizer=best_cnn_hyperparameters['Optimizer'], 
                                                 loss='categorical_crossentropy', 
                                                 metrics=['accuracy'])
print(model_cnn.summary())

history_cnn= model_cnn.fit(train_x, train_y, epochs=50, batch_size=32, validation_data=(dev_x, dev_y), callbacks=callback)

In [ ]:
plt.plot(history.history['loss'][5:], label='Training Loss')
plt.plot(history.history['val_loss'][5:], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 在训练集上计算 R² 分数
y_train_pred = model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

# 在验证集上计算 R² 分数
y_test_pred = model.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)

# 打印 R² 分数
print(f'R² 分数 - 训练集: {r2_train:.4f}')
print(f'R² 分数 - 验证集: {r2_test:.4f}')

In [ ]:
# 绘制真实值和预测值的对比图
plt.scatter(y_test, y_test_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
plt.title('cnn')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()